In [1]:
import pandas as pd
import numpy as np
import requests
import json
import matplotlib.pyplot as plt
import yfinance as yf

In [2]:
def financial_ratios(ticker):
    stock = yf.download(tickers=ticker, auto_adjust=True)
    apikey = '3WZH46PUYUQ32UBX'
    url_IS = f'https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={ticker}&apikey={apikey}'
    url_BS = f'https://www.alphavantage.co/query?function=BALANCE_SHEET&symbol={ticker}&apikey={apikey}'
    url_CF = f'https://www.alphavantage.co/query?function=CASH_FLOW&symbol={ticker}&apikey={apikey}'

    r_IS = requests.get(url=url_IS)
    r_BS = requests.get(url=url_BS)
    r_CF = requests.get(url=url_CF)
    js_IS = r_IS.json()['annualReports']
    js_BS = r_BS.json()['annualReports']
    js_CF = r_CF.json()['annualReports']
    df_IS = pd.DataFrame(js_IS)
    df_BS = pd.DataFrame(js_BS)
    df_CF = pd.DataFrame(js_CF)
    data = pd.concat([df_IS, df_BS, df_CF], axis=1)
    data = data.T.drop_duplicates(keep='last').T
    data['fiscalDateEnding'] = pd.to_datetime(data['fiscalDateEnding'], format='%Y/%m/%d')
    data.set_index(data['fiscalDateEnding'], inplace=True)
    data.sort_index(ascending=True, inplace=True)
    data.drop(columns=['fiscalDateEnding', 'reportedCurrency'], inplace=True)
    data = data.replace('None', 0)
    data = data.astype(float)
    financial_data = data.join(stock['Close'], how='inner')
    financial_data.rename(columns={'Close': 'price'}, inplace=True)
    return financial_data

In [3]:
data = financial_ratios('MELI')
data

[*********************100%***********************]  1 of 1 completed


,grossProfit,totalRevenue,costOfRevenue,costofGoodsAndServicesSold,sellingGeneralAndAdministrative,researchAndDevelopment,operatingExpenses,netInterestIncome,interestIncome,interestExpense,...,paymentsForRepurchaseOfEquity,dividendPayoutCommonStock,dividendPayoutPreferredStock,proceedsFromIssuanceOfCommonStock,proceedsFromIssuanceOfLongTermDebtAndCapitalSecuritiesNet,proceedsFromRepurchaseOfEquity,changeInCashAndCashEquivalents,changeInExchangeRate,netIncome,price
2018-12-31,6.970080e+08,1.439653e+09,7.426450e+08,7.426450e+08,6.202170e+08,146273000.0,7.664900e+08,-14210000.0,42039000.0,56249000.0,...,0.0,6624000.0,0.0,0.000000e+00,2.368730e+08,0.000000e+00,1.673300e+08,0.0,-36585000.0,292.850006
2019-12-31,1.102123e+09,2.651821e+09,2.298988e+09,1.194191e+09,1.031477e+09,223807000.0,1.255284e+09,-1798000.0,113523000.0,1798000.0,...,720000.0,2844000.0,2844000.0,1.867215e+09,6.298910e+08,1.965183e+09,1.024313e+09,0.0,-171999000.0,571.940002
2020-12-31,1.709000e+09,4.381090e+09,3.598291e+09,2.265000e+09,1.229044e+09,353000000.0,1.581000e+09,-2813000.0,102767000.0,2813000.0,...,54085000.0,3356000.0,3000000.0,0.000000e+00,2.396717e+09,1.911818e+09,1.172637e+09,0.0,-1000000.0,1675.219971
2021-12-31,3.005000e+09,7.527889e+09,6.250967e+09,4.064000e+09,1.974021e+09,590000000.0,2.564000e+09,-4953000.0,137962000.0,4953000.0,...,485886000.0,0.0,0.0,1.520000e+09,9.261703e+09,1.033659e+09,1.293493e+09,0.0,83000000.0,1348.400024


In [4]:
ratios = data.loc[:, ['totalCurrentAssets', 'totalCurrentLiabilities', 'inventory', 'netIncome', 'commonStockSharesOutstanding', 'price', 'totalLiabilities']]
ratios['']
ratios

,totalCurrentAssets,totalCurrentLiabilities,inventory,netIncome,commonStockSharesOutstanding,price,totalLiabilities
2018-12-31,1.511123e+09,1.166224e+09,4612000.0,-36585000.0,45202859.0,292.850006,1.902819e+09
2019-12-31,3.788856e+09,1.752320e+09,8626000.0,-171999000.0,49709955.0,571.940002,2.699728e+09
2020-12-31,5.346807e+09,3.635880e+09,118140000.0,-1000000.0,49869727.0,1675.219971,4.874754e+09
2021-12-31,8.175000e+09,5.837000e+09,253000000.0,83000000.0,50418980.0,1348.400024,8.570000e+09
